# Import Assignments From a SQLite Database
In this example, assignments stored in a SQLite database, representing sidewalks that need to be repaired, will be imported into a Workforce Project as assignments.

In [ ]:
import sqlite3
from datetime import datetime
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps import workforce
from arcgis.geocoding import geocode

### Connect to Organization and Get the Project
Let's connect to ArcGIS Online and find the project.

In [ ]:
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("1f7b42024da544f6b1e557889e858ac6")
project = workforce.Project(item)

### Get the Sidewalk Repair Assignment Type and the Dispatcher to Use
Let's get the "Sidewalk" assignment type and the dispatcher from the Project.

In [ ]:
sidewalk_repair = project.assignment_types.get(name="Sidewalk Repair")
dispatcher = project.dispatchers.get(user_id="workforce_scripts")

### View All Assignments in SQLite
Let's connect to the "work_orders" database and query all work orders from the "work_orders" table.

In [ ]:
connection = sqlite3.connect("work_orders")
df = pd.read_sql_query("select * from work_orders", connection)
df

### Extract Specific Assignments to Add
Notice how some orders have a status of "Completed" and how other orders have a status of "Backlog". Let's only query work orders that are in the backlog.

In [ ]:
df = pd.read_sql_query("select * from work_orders where status='Backlog'", connection)
df

### Create An Assignment For Each Row
For each assignment, first geocode the address to get the x,y location (in WGS 84 Web Mercator) of the assignment. Then supply additional attributes. Finally use the batch_add method to add multiple assignments at once (this is faster than using the add method since validation is performed once for all assignments).

In [ ]:
assignments = []
for index, row in df.iterrows():
    geometry = geocode(row["address"], out_sr=3857)[0]["location"]
    assignments.append(
        workforce.Assignment(
            project,
            geometry=geometry,
            location=row["address"],
            description=row["description"],
            priority=int(row["priority"]),
            work_order_id=row["id"],
            assignment_type=sidewalk_repair,
            dispatcher=dispatcher,
            status="unassigned",
            due_date=pd.to_datetime(row["due_date"])
        )
    )
project.assignments.batch_add(assignments)

### Verify the assignments on the map
Let's verify that these sidewalk assignments were added.

In [ ]:
webmap = gis.map("San Diego", zoomlevel=14)
webmap.add_layer(project.assignments_layer)
webmap